In [1]:
import urllib
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import openapi

In [ ]:
import requests

# ✅ API 키 및 URL 설정
s_key = 'PwfNqgN7hal2oSaDMz96GOgjRcDcPrZUybaows5KKV1Xa0uMKVhAbb/e2y78IJMwQ+mHPTsZgjBVqkk54niRUA=='
url = 'http://apis.data.go.kr/B551172/Kidney03/kiJobByType'

# ✅ 기본 파라미터 설정 (numOfRows를 정수로 변경)
params = {
    'serviceKey': s_key,
    'type': 'json',
    'numOfRows': 100,  # 🔹 문자열이 아닌 정수로 변경
    'centerNm': '국립암센터',
    'fromYear': '2010',
    'toYear': '2019',
    'pageNo': 1
}

all_data = []

# 1️⃣ 먼저 첫 번째 요청을 보내서 totalCount 확인
response = requests.get(url, params=params)

if response.status_code == 200:
    resp_dict = response.json()
    
    # 🔹 totalCount가 없는 경우 대비하여 기본값 0 설정
    total_count = int(resp_dict.get('totalCount', 0))  
    items_per_page = params['numOfRows']  
    total_pages = (total_count // items_per_page) + (1 if total_count % items_per_page else 0)

    print(f"📌 총 데이터 개수: {total_count}, 가져올 페이지 수: {total_pages}")

    # 2️⃣ total_pages 만큼 반복하여 모든 데이터를 가져오기
    for i in range(1, total_pages + 1):
        params['pageNo'] = i  
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            resp_dict = response.json()
            items = resp_dict.get('items', [])  # 🔹 데이터가 없을 경우 빈 리스트 반환
            all_data.extend(items)
        else:
            print(f"❌ 데이터 요청 실패 (페이지 {i})")

    print(f"✅ 총 {len(all_data)}개의 데이터 수집 완료!")

else:
    print("❌ API 요청 실패")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df_job = df_job.iloc[:, [1,2,3,4,5,-2]]
df_job

In [ ]:
df_job.rename(columns={'centerNm':'센터 이름', 'ptAge':'나이', 'ptSexCd':'성별', 'statsTrgtNm':'직업', 'ptCntNmvl':'환자수'}, inplace=True)
df_job

In [ ]:
df_job['환자수'] = pd.to_numeric(df_job['환자수'], errors='coerce')

df_job['환자수'] = df_job['환자수'].fillna(0).astype(int)

job_count = df_job.groupby(df_job['직업'].fillna('무응답'))['환자수'].sum()

print(job_count)

In [ ]:
import matplotlib.font_manager as fm

# 한글 폰트 설정 (예: 맑은 고딕)
plt.rcParams['font.family'] = 'Malgun Gothic'

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 파이차트 그리기
plt.figure(figsize=(6, 6))
plt.pie(job_count.values, labels=job_count.index, autopct='%1.1f%%', startangle=140)
plt.title('직업 분포')
plt.axis('equal')
plt.show()